<a href="https://colab.research.google.com/github/yejikwon7/DeepLearning/blob/main/final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import os

# 데이터셋 경로를 여기에 입력
dataset_path = "/root/.cache/kagglehub/datasets/sautkin/imagenet1k1"

# 파일 해시값 계산 함수
def calculate_file_hash(filepath):
    with open(filepath, "rb") as f:
        file_hash = hashlib.md5()
        while chunk := f.read(8192):
            file_hash.update(chunk)
    return file_hash.hexdigest()

# 경로 내 파일 해시값 확인
hashes = {}
for dirpath, dirnames, filenames in os.walk(dataset_path):
    for file in filenames:
        file_path = os.path.join(dirpath, file)
        file_hash = calculate_file_hash(file_path)
        if file_hash in hashes:
            print(f"Duplicate File Found: {file_path} and {hashes[file_hash]}")
        else:
            hashes[file_hash] = file_path

import shutil

# 데이터셋 경로 삭제
shutil.rmtree(dataset_path)
print(f"Deleted folder: {dataset_path}")

Duplicate File Found: /root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2/00578/380489571364188.jpg and /root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2/00885/017351765019286.jpg
Duplicate File Found: /root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2/00884/555296479777531.jpg and /root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2/00687/038192301570222.jpg
Duplicate File Found: /root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2/00635/0876804377700111.jpg and /root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2/00635/193898156016692.jpg
Duplicate File Found: /root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2/00530/8808252596861657.jpg and /root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2/00531/308497323199823.jpg
Duplicate File Found: /root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2/00530/8104005986347684.jpg and /root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2/005

In [ ]:
import torch
torch.cuda.empty_cache()  # GPU 캐시 메모리 비우기
torch.cuda.reset_peak_memory_stats()

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kwonyj7","key":"4ecbdea9e787374fd05bb00127163939"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sautkin/imagenet1k1")

print("Path to dataset files:", path)

100%|██████████| 11.0G/11.0G [01:59<00:00, 98.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image, ImageDraw
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# # 중간 저장
# def save_checkpoint(model, optimizer, epoch, path="checkpoint.pth"):
#     checkpoint = {
#         "model_state_dict": model.state_dict(),
#         "optimizer_state_dict": optimizer.state_dict(),
#         "epoch": epoch
#     }
#     torch.save(checkpoint, path)
#     print(f"Checkpoint saved at epoch {epoch} to {path}")

In [ ]:
# # 중간 저장 불러오기
# def load_checkpoint(model, optimizer, path="checkpoint.pth"):
#     checkpoint = torch.load(path)
#     model.load_state_dict(checkpoint["model_state_dict"])
#     optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
#     start_epoch = checkpoint["epoch"] + 1
#     print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")
#     return start_epoch

In [ ]:
# hyper-parameters
lr = 0.001
epochs = 30
batch_size = 64

In [ ]:
# YOLOv5 모델 불러오기 (PyTorch Hub)
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-17 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
# 데이터셋 로드 및 전처리
imagenet_data_path = "/root/.cache/kagglehub/datasets/sautkin/imagenet1k1/versions/2"
imagenet_transforms = transforms.Compose([
    transforms.Resize((640, 640)),  # YOLOv5가 요구하는 입력 크기로 조정
    transforms.ToTensor(),          # 이미지를 Tensor로 변환
])

In [ ]:
# 원본 데이터셋 로드
imagenet_dataset = datasets.ImageFolder(imagenet_data_path, transform=imagenet_transforms)

# 데이터셋 랜덤 샘플링
random.seed(42)  # 재현성을 위해 시드 설정
sample_indices = random.sample(range(len(imagenet_dataset)), 500)
sampled_dataset = torch.utils.data.Subset(imagenet_dataset, sample_indices)

In [ ]:
# train / test 8:2 분할
train_indices, test_indices = train_test_split(range(len(sampled_dataset)), test_size=0.2, random_state=42)

# train / validation 8:2 분할
train_indices, val_indices = train_test_split(train_indices, test_size=0.2, random_state=42)

In [ ]:
# 데이터셋
train_dataset = torch.utils.data.Subset(sampled_dataset, train_indices)
val_dataset = torch.utils.data.Subset(sampled_dataset, val_indices)
test_dataset = torch.utils.data.Subset(sampled_dataset, test_indices)

In [ ]:
# dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

# 데이터 로드 확인
print(f"샘플링된 데이터셋 크기: {len(sampled_dataset)}")
print(f"훈련 데이터 크기: {len(train_dataset)}")
print(f"검증 데이터 크기: {len(val_dataset)}")
print(f"테스트 데이터 크기: {len(test_dataset)}")

In [ ]:
import torch.nn as nn

# ResNet50 모델 불러오기
resnet_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

resnet_model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)

# Dropout을 추가한 Fully Connected Layer
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(num_ftrs, len(imagenet_dataset.classes))  # 최종 분류 레이어
)

resnet_model = resnet_model.cuda()

Using cache found in /root/.cache/torch/hub/pytorch_vision_main


In [ ]:
# 최적화 함수 및 손실 함수 설정
optimizer = torch.optim.Adam(resnet_model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [ ]:
# ImageNet 클래스 이름 매핑
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
imagenet_classes = {idx: entry.strip() for idx, entry in enumerate(open("imagenet_classes.txt"))}

--2024-12-17 06:04:42--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.001s  

2024-12-17 06:04:42 (13.2 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [ ]:
# YOLOv5 결과 출력 함수
def visualize_yolo_results(image, detections):
    draw = ImageDraw.Draw(image)
    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection[:6]
        draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
        draw.text((x1, y1), f"Class: {int(cls)}, Conf: {conf:.2f}", fill="red")
    return image

In [ ]:
# ResNet 결과 출력 함수
def visualize_resnet_results(image, outputs):
    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
    top5_prob, top5_classes = torch.topk(probabilities, 5)
    draw = ImageDraw.Draw(image)
    text = "\n".join([f"{imagenet_classes[idx]}: {prob:.6f}" for idx, prob in zip(top5_classes, top5_prob)])
    draw.text((10, 10), text, fill="blue")
    return image

In [ ]:
# 학습률 갱신 함수
def adjust_learning_rate(optimizer, epoch, lr, val_accuracy):
    if val_accuracy > 0.9:  # Accuracy가 90% 이상이면 학습률 감소
        lr *= 0.5
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    print(f"Epoch {epoch}: Updated Learning Rate = {lr:.6f}")
    return lr

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, criterion, epochs, class_names, save_path='best_resnet_model.pth'):
    best_val_loss = float('inf')
    train_losses, val_losses = [], []

    for epoch in range(epochs):
        # Training Phase
        model.train()
        running_train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item() * inputs.size(0)

        epoch_train_loss = running_train_loss / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)

        # Validation Phase
        model.eval()
        running_val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.cuda(), labels.cuda()
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_val_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                correct += predicted.eq(labels).sum().item()
                total += labels.size(0)

        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)
        val_accuracy = 100. * correct / total

        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {epoch_train_loss:.4f}, Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

        # Save the best model
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            torch.save(model.state_dict(), save_path)
            print(f"Best model saved with Validation Loss: {epoch_val_loss:.4f}")

    # 손실 그래프 출력
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, epochs + 1), train_losses, label="Train Loss")
    plt.plot(range(1, epochs + 1), val_losses, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.show()

    # 학습 결과 이미지 출력
    print("Displaying training results with sample images:")
    visualize_train_results(model, val_loader, class_names, num_images=5)

In [ ]:
# 학습 이미지 결과 시각화 함수
def visualize_train_results(model, train_loader, class_names, num_images=5):
    model.eval()
    images_shown = 0

    plt.figure(figsize=(15, 10))
    with torch.no_grad():
        for inputs, labels in train_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, predicted = outputs.max(1)

            for i in range(inputs.size(0)):
                if images_shown >= num_images:
                    break

                image = inputs[i].cpu().permute(1, 2, 0).numpy()
                image = (image * 0.229 + 0.485)  # Unnormalize
                label = class_names[labels[i].item()]
                prediction = class_names[predicted[i].item()]

                plt.subplot(1, num_images, images_shown + 1)
                plt.imshow(image)
                plt.title(f"True: {label}\nPred: {prediction}")
                plt.axis('off')
                images_shown += 1

            if images_shown >= num_images:
                break

    plt.tight_layout()
    plt.show()

In [ ]:
class_names = imagenet_dataset.classes

# 학습 실행
train_model(
    resnet_model,
    train_loader,
    val_loader,
    optimizer,
    criterion,
    epochs,
    class_names,
    save_path='best_resnet_model.pth'
)

In [ ]:
# 테스트 함수
def test_model(model, test_loader, criterion, class_names, num_images=5):
    test_loss = 0.0
    correct = 0
    total = 0
    model = model.cuda()
    model.eval()

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / total

    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

    # 테스트 결과 시각화
    visualize_test_results(model, test_loader, class_names, num_images)

    return test_loss, test_accuracy

# 테스트 결과 시각화 함수
def visualize_test_results(model, test_loader, class_names, num_images=5):
    model = model.cuda()
    model.eval()
    images_shown = 0

    plt.figure(figsize=(15, 10))
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, predicted = outputs.max(1)

            for i in range(inputs.size(0)):
                if images_shown >= num_images:
                    break

                image = inputs[i].cpu().permute(1, 2, 0).numpy()
                image = (image * 0.229 + 0.485)  # Unnormalize
                label = class_names[labels[i].item()]
                prediction = class_names[predicted[i].item()]

                plt.subplot(1, num_images, images_shown + 1)
                plt.imshow(image)
                plt.title(f"True: {label}\nPred: {prediction}")
                plt.axis('off')
                images_shown += 1

            if images_shown >= num_images:
                break

    plt.tight_layout()
    plt.show()

In [ ]:
# test 실행
test_model(
    resnet_model,
    test_loader,
    criterion,
    class_names=imagenet_dataset.classes
)